In [1]:
import os
import io
import sys
import time
import smtplib
from email.message import EmailMessage
import pandas as pd
from tqdm.notebook import tqdm
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

## Authentication

In [2]:
# Defining scope for spreadsheets
SCOPES = ["https://www.googleapis.com/auth/spreadsheets","https://www.googleapis.com/auth/drive"]

In [3]:
def authenticate_driveAPI(SCOPES):
    # Authentication process
    credentials = None
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json", scopes=SCOPES)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("/Users/admin/Documents/Credentials/Google Sheets/OAuth-Credentials.json", SCOPES)
            credentials = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(credentials.to_json())
    
    try:
        # Build the service
        drive_service = build("drive", "v3", credentials=credentials)

        return drive_service
    except HttpError as error:
        print(error)

In [5]:
def authenticate_sheetsAPI(SCOPES):
    # Authentication process
    credentials = None
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json", scopes=SCOPES)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("/Users/admin/Documents/Credentials/Google Sheets/OAuth-Credentials.json", SCOPES)
            credentials = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(credentials.to_json())
    
    try:
        # Build the service
        sheets_service = build("sheets", "v4", credentials=credentials)
        #sheets_service = sheets_service.spreadsheets()

        return sheets_service
    except HttpError as error:
        print(error)

In [4]:
drive = authenticate_driveAPI(SCOPES)
drive.files()

In [6]:
sheets = authenticate_sheetsAPI(SCOPES)
sheets

In [216]:
drive, sheets = authenticate_APIs(SCOPES)

In [38]:
result = sheets.values().get(spreadsheetId=SPREADSHEET_ID, range="A:Z", majorDimension="ROWS").execute()
values = result.get("values", [])
values

[['DEPARTMENT OF COMPUTER SCIENCE AND ENGINEERING'],
 [],
 ['Academic Year: Apr23   to Oct 23'],
 ['Class Teacher : Asha Kumari A'],
 ['Result Sheet'],
 [],
 ['S.NO',
  'USN',
  'NAME',
  '21CSL481(WP)                        INT        EXT',
  '',
  'Total',
  '21MATCS41(MAT)                        INT          EXT',
  '',
  'Total',
  '21CS42(DAA)                                             INT          EXT',
  '',
  'Total',
  '21CS43(MCES)                                        INT      EXT',
  '',
  'Total',
  '21CS44(OS) INT      EXT',
  '',
  'Total',
  '21BE45(BE)                   INT        EXT',
  '',
  'Total',
  '21CSL46(PPL)     INT         EXT',
  '',
  'Total',
  '21KBK47/21KSK47                                   INT         EXT'],
 ['1',
  '1OX21CS116',
  'ROHAN VATS',
  '50',
  '50',
  '100',
  '46',
  '12',
  '58',
  '47',
  '22',
  '69',
  '43',
  '22',
  '65',
  '38',
  '14',
  '52',
  '37',
  '19',
  '56',
  '50',
  '50',
  '100',
  '49',
  '45'],
 ['2',
  '1OX21CS

In [31]:
drive.files()

In [226]:
# Call the Drive v3 API
results = (
        drive.files()
        .list(pageSize=10, fields="files(id, name)")
        .execute()
    )

In [227]:
results

{'files': [{'id': '1rZUZaa9R0XxXU8OS0deKVXPWg3vp_SIBl5fAOiFMDoU',
   'name': 'CSE students personal details'},
  {'id': '1MAWB8k27Xw_elRJTvaQ8LY0UGzuQZxod', 'name': 'Class 4C'},
  {'id': '1ob91oVztOJF1nM1DkyahjJDKMrZgcN4T', 'name': '1OX21CS171.pdf'},
  {'id': '1b1-Ild8X4JYbMksXBNvjPWe_wAEhnSz4', 'name': '1OX21CS170.pdf'},
  {'id': '1f8-h1QQZT5N6psL9dE6pxaTYWSFHNHJx', 'name': '1OX21CS169.pdf'},
  {'id': '1tebH2EV1Fk93q3Br2XBULBf1Nu87CBbL', 'name': '1OX21CS168.pdf'},
  {'id': '1JPr-YbgZnXWWGhy6AD5OMn67z4OEPn6q', 'name': '1OX21CS167.pdf'},
  {'id': '1lFzt5Qew8FZpvs3b6jBoTm5kWtv53RsW', 'name': '1OX21CS166.pdf'},
  {'id': '1zUh_ZZf25ondj9FKKPBgSleD4gOSiOST', 'name': '1OX21CS165.pdf'},
  {'id': '1qIl_AN_9omVTHptUMEV1rYpnwuOc-Quq', 'name': '1OX21CS164.pdf'}]}

In [3]:
def extract_data_from_sheet():
    # Authentication process
    credentials = None
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json", scopes=SCOPES)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("/Users/admin/Documents/Credentials/Google Sheets/OAuth-Credentials.json", SCOPES)
            credentials = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(credentials.to_json())

    try:
        # Build the service
        service = build("sheets", "v4", credentials=credentials)
        sheets = service.spreadsheets()

        # Fetch data from the spreadsheet row-wise
        result = sheets.values().get(spreadsheetId=SPREADSHEET_ID, range="A:Z", majorDimension="ROWS").execute()
        values = result.get("values", [])

        # Convert data to a Pandas DataFrame
        df = pd.DataFrame(values)

        return df
    except HttpError as error:
        print(error)

In [4]:
dataframe = extract_data_from_sheet()
dataframe

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,DEPARTMENT OF COMPUTER SCIENCE AND ENGINEERING,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Academic Year: Apr23 to Oct 23,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Class Teacher : Asha Kumari A,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Result Sheet,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,,1OX21CS120,S SARVESHWAR,692,87,VI,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
92,,1OX21CS123,SAHANA R,691,86.36,VII,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
93,,1OX21CS145,SRIMATHI J,684,85.5,VIII,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
94,,1OX21CS152,SYED IBRAHIM,684,85.5,IX,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [5]:
results = dataframe.copy()
results

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,DEPARTMENT OF COMPUTER SCIENCE AND ENGINEERING,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Academic Year: Apr23 to Oct 23,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Class Teacher : Asha Kumari A,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Result Sheet,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,,1OX21CS120,S SARVESHWAR,692,87,VI,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
92,,1OX21CS123,SAHANA R,691,86.36,VII,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
93,,1OX21CS145,SRIMATHI J,684,85.5,VIII,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
94,,1OX21CS152,SYED IBRAHIM,684,85.5,IX,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [9]:
results = results[6:]
results

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
6,S.NO,USN,NAME,21CSL481(WP) INT ...,,Total,21MATCS41(MAT) INT ...,,Total,21CS42(DAA) ...,...,,Total,21BE45(BE) INT EXT,,Total,21CSL46(PPL) INT EXT,,Total,21KBK47/21KSK47 ...,None
7,1,1OX21CS116,ROHAN VATS,50,50,100,46,12,58,47,...,14,52,37,19,56,50,50,100,49,45
8,2,1OX21CS117,ROJA MK,49,47,96,49,43,92,45,...,24,72,46,37,83,50,40,90,48,46
9,3,1OX21CS118,ROOPA R,50,50,100,50,34,84,46,...,44,92,50,38,88,50,43,93,48,48
10,4,1OX21CS119,RUSHIKESH B KATTIMANI,50,46,96,39,32,71,37,...,37,79,43,31,74,50,45,95,44,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,,1OX21CS120,S SARVESHWAR,692,87,VI,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
92,,1OX21CS123,SAHANA R,691,86.36,VII,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
93,,1OX21CS145,SRIMATHI J,684,85.5,VIII,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
94,,1OX21CS152,SYED IBRAHIM,684,85.5,IX,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [16]:
results = results[:66]
results

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
6,S.NO,USN,NAME,21CSL481(WP) INT ...,,Total,21MATCS41(MAT) INT ...,,Total,21CS42(DAA) ...,...,,Total,21BE45(BE) INT EXT,,Total,21CSL46(PPL) INT EXT,,Total,21KBK47/21KSK47 ...,None
7,1,1OX21CS116,ROHAN VATS,50,50,100,46,12,58,47,...,14,52,37,19,56,50,50,100,49,45
8,2,1OX21CS117,ROJA MK,49,47,96,49,43,92,45,...,24,72,46,37,83,50,40,90,48,46
9,3,1OX21CS118,ROOPA R,50,50,100,50,34,84,46,...,44,92,50,38,88,50,43,93,48,48
10,4,1OX21CS119,RUSHIKESH B KATTIMANI,50,46,96,39,32,71,37,...,37,79,43,31,74,50,45,95,44,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,61,1OX22CS420,SINDHU T R,46,47,93,35,18,53,34,...,26,67,32,29,61,46,38,84,37,29
68,62,1OX22CS421,SPOORTHI HOSAMANI,46,47,93,41,19,60,37,...,28,65,38,31,69,45,43,88,48,46
69,63,1OX22CS422,SUNIL TEGGI,45,45,90,39,9,48,34,...,19,54,38,38,76,43,38,81,44,47
70,64,1OX22CS423,UDAY KIRAN G,45,45,90,44,36,80,36,...,20,62,33,26,59,44,43,87,41,38


In [18]:
results[:1]

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
6,S.NO,USN,NAME,21CSL481(WP) INT ...,,Total,21MATCS41(MAT) INT ...,,Total,21CS42(DAA) ...,...,,Total,21BE45(BE) INT EXT,,Total,21CSL46(PPL) INT EXT,,Total,21KBK47/21KSK47 ...,None


In [26]:
results[[3]].iloc[0]

3    21CSL481(WP)                        INT       ...
Name: 6, dtype: object

In [27]:
results.iloc[0]

0                                                  S.NO
1                                                   USN
2                                                  NAME
3     21CSL481(WP)                        INT       ...
4                                                      
5                                                 Total
6     21MATCS41(MAT)                        INT     ...
7                                                      
8                                                 Total
9     21CS42(DAA)                                   ...
10                                                     
11                                                Total
12    21CS43(MCES)                                  ...
13                                                     
14                                                Total
15                              21CS44(OS) INT      EXT
16                                                     
17                                              

In [28]:
results.columns = results.iloc[0]
results

6,S.NO,USN,NAME,21CSL481(WP) INT EXT,,Total,21MATCS41(MAT) INT EXT,,Total,21CS42(DAA) INT EXT,...,,Total,21BE45(BE) INT EXT,,Total,21CSL46(PPL) INT EXT,,Total,21KBK47/21KSK47 INT EXT,None
6,S.NO,USN,NAME,21CSL481(WP) INT ...,,Total,21MATCS41(MAT) INT ...,,Total,21CS42(DAA) ...,...,,Total,21BE45(BE) INT EXT,,Total,21CSL46(PPL) INT EXT,,Total,21KBK47/21KSK47 ...,None
7,1,1OX21CS116,ROHAN VATS,50,50,100,46,12,58,47,...,14,52,37,19,56,50,50,100,49,45
8,2,1OX21CS117,ROJA MK,49,47,96,49,43,92,45,...,24,72,46,37,83,50,40,90,48,46
9,3,1OX21CS118,ROOPA R,50,50,100,50,34,84,46,...,44,92,50,38,88,50,43,93,48,48
10,4,1OX21CS119,RUSHIKESH B KATTIMANI,50,46,96,39,32,71,37,...,37,79,43,31,74,50,45,95,44,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,61,1OX22CS420,SINDHU T R,46,47,93,35,18,53,34,...,26,67,32,29,61,46,38,84,37,29
68,62,1OX22CS421,SPOORTHI HOSAMANI,46,47,93,41,19,60,37,...,28,65,38,31,69,45,43,88,48,46
69,63,1OX22CS422,SUNIL TEGGI,45,45,90,39,9,48,34,...,19,54,38,38,76,43,38,81,44,47
70,64,1OX22CS423,UDAY KIRAN G,45,45,90,44,36,80,36,...,20,62,33,26,59,44,43,87,41,38


In [29]:
results.columns

Index([                                                                    'S.NO',
                                                                            'USN',
                                                                           'NAME',
                             '21CSL481(WP)                        INT        EXT',
                                                                               '',
                                                                          'Total',
                         '21MATCS41(MAT)                        INT          EXT',
                                                                               '',
                                                                          'Total',
       '21CS42(DAA)                                             INT          EXT',
                                                                               '',
                                                                          'Total',
    

6,Total,Total,Total,Total,Total,Total,Total
6,Total,Total,Total,Total,Total,Total,Total
7,100,58,69,65,52,56,100
8,96,92,70,87,72,83,90
9,100,84,77,74,92,88,93
10,96,71,68,67,79,74,95
...,...,...,...,...,...,...,...
67,93,53,63,74,67,61,84
68,93,60,70,70,65,69,88
69,90,48,73,76,54,76,81
70,90,80,72,77,62,59,87


In [32]:
SPREADSHEET_ID = "1hIg56laGnWo2EEQbAHr1Wn9wP3NhOPydab0DMtOZ86Y"

In [36]:
def write_dataframe_to_sheet(df):
    # Authenticate with Google Sheets API
    credentials = None
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("/Users/admin/Documents/Credentials/Google Sheets/OAuth-Credentials.json", SCOPES)
            credentials = flow.run_local_server(port=0)

    # Initialize the Google Sheets service
    service = build("sheets", "v4", credentials=credentials)

    # Open the spreadsheet
    sheet = service.spreadsheets()

    # Define the range where you want to write the DataFrame (e.g., "Sheet1!A1")
    range_name = "Sheet1!A1"

    # Convert the DataFrame to a list of lists (values)
    values = df.values.tolist()

    # Update the values in the spreadsheet
    body = {"values": values}
    result = sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=range_name, valueInputOption="RAW", body=body).execute()

    print("DataFrame successfully written to Google Sheets.")

In [37]:
write_dataframe_to_sheet(results)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=424851802072-hr57qs09v4fp0bniln2g0clg01qe9nb4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54188%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=KqnYhAqWaxUnAzUHmcmMKzX00IMgzp&access_type=offline
DataFrame successfully written to Google Sheets.


## Table Pre-processing and Fornatting

In [38]:
import fitz

In [40]:
doc = fitz.open("PDFs/VTU Result 4th sem.pdf") # open a document
for page in doc: # iterate the document pages
  text = page.get_text() # get plain text encoded as UTF-8
text

'INTERNSHIP\n ನಾಮಕರಣ / ಸಂಕ್ಷೇಪಣಗಳು\nNomenclature / Abbreviations\nP -> PASS\nF -> FAIL\nA -> ABSENT\nW -> WITHHELD\nX, NE -> NOT ELIGIBLE\nNote :\n1) Results of some subjects of some students are not appearing due to reasons such as,\na) CIE not Available\nb) SEE not available\nbecause of technical reasons etc, however they will be updated shortly.\n2) Withheld results to be announced later.\n\ue012\nಕುಲಸಚಿವರು (ಮೌಲ್ಯಮಾಪನ)\nREGISTRAR (EVALUATION)\nಸಹಿ/-\nSd/-\n'

In [39]:
import fitz
#from pprint import pprint

doc = fitz.open("PDFs/VTU Result 4th sem.pdf") # open document
page = doc[0] # get the 1st page of the document
tabs = page.find_tables() # locate and extract any tables on page
print(f"{len(tabs.tables)} found on {page}") # display number of found tables
if tabs.tables:  # at least one table found?
   print(tabs[0].extract())  # print content of first table

2 found on page 0 of PDFs/VTU Result 4th sem.pdf
[['University Seat Number', ': 1OX21CS144'], ['Student Name', ': SRIKAR V']]


In [40]:
print(tabs[1].extract())

[['Subject\nCode', 'Subject Name', 'Internal\nMarks', 'External\nMarks', 'Total', 'Result', 'Announced\n/ Updated\non'], ['1MATCS41', 'MATHEMATICAL\nFOUNDATIONS FOR\nCOMPUTING,\nPROBABILITY &\nSTATISTICS', '47', '44', '91', 'P', '2023-11-03'], ['21CSL481', 'WEB\nPROGRAMMING', '50', '50', '100', 'P', '2023-11-03'], ['21CS42', 'DESIGN AND\nANALYSIS OF\nALGORITHMS', '49', '40', '89', 'P', '2023-11-03'], ['21CS43', 'MICROCONTROLLER\nAND EMBEDDED\nSYSTEM', '49', '40', '89', 'P', '2023-11-03'], ['21CS44', 'OPERATING SYSTEM', '44', '39', '83', 'P', '2023-11-03'], ['21BE45', 'BIOLOGY FOR\nENGINEERS', '45', '35', '80', 'P', '2023-11-03'], ['21CSL46', 'PYTHON\nPROGRAMMING\nLABORATORY', '50', '49', '99', 'P', '2023-11-03'], ['21KSK47', 'SAMSKRUTIKA\nKANNADA', '39', '39', '78', 'P', '2023-11-03'], ['21UH49', 'UNIVERSAL HUMAN\nVALUES AND\nPROFESSIONAL\nETHICS', '47', '37', '84', 'P', '2023-11-03']]


In [41]:
check = pd.DataFrame(tabs[1].extract())

In [42]:
check#.transpose()

,0,1,2,3,4,5,6
0,Subject\nCode,Subject Name,Internal\nMarks,External\nMarks,Total,Result,Announced\n/ Updated\non
1,1MATCS41,"MATHEMATICAL\nFOUNDATIONS FOR\nCOMPUTING,\nPRO...",47,44,91,P,2023-11-03
2,21CSL481,WEB\nPROGRAMMING,50,50,100,P,2023-11-03
3,21CS42,DESIGN AND\nANALYSIS OF\nALGORITHMS,49,40,89,P,2023-11-03
4,21CS43,MICROCONTROLLER\nAND EMBEDDED\nSYSTEM,49,40,89,P,2023-11-03
5,21CS44,OPERATING SYSTEM,44,39,83,P,2023-11-03
6,21BE45,BIOLOGY FOR\nENGINEERS,45,35,80,P,2023-11-03
7,21CSL46,PYTHON\nPROGRAMMING\nLABORATORY,50,49,99,P,2023-11-03
8,21KSK47,SAMSKRUTIKA\nKANNADA,39,39,78,P,2023-11-03
9,21UH49,UNIVERSAL HUMAN\nVALUES AND\nPROFESSIONAL\nETHICS,47,37,84,P,2023-11-03


In [129]:
example = pd.DataFrame(columns=['USN','Name'])
example

,USN,Name


In [48]:
details = tabs[0].extract()
details

[['University Seat Number', ': 1OX21CS144'], ['Student Name', ': SRIKAR V']]

In [52]:
details[0][1][2:]

'1OX21CS144'

In [53]:
details[1][1][2:]

'SRIKAR V'

In [54]:
details = [details[0][1][2:],details[1][1][2:]]
details

['1OX21CS144', 'SRIKAR V']

In [130]:
example.loc[0] = details

In [131]:
example

,USN,Name
0,1OX21CS144,SRIKAR V


In [61]:
check = check.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x)
check

,0,1,2,3,4,5,6
0,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on
1,1MATCS41,"MATHEMATICAL FOUNDATIONS FOR COMPUTING, PROBAB...",47,44,91,P,2023-11-03
2,21CSL481,WEB PROGRAMMING,50,50,100,P,2023-11-03
3,21CS42,DESIGN AND ANALYSIS OF ALGORITHMS,49,40,89,P,2023-11-03
4,21CS43,MICROCONTROLLER AND EMBEDDED SYSTEM,49,40,89,P,2023-11-03
5,21CS44,OPERATING SYSTEM,44,39,83,P,2023-11-03
6,21BE45,BIOLOGY FOR ENGINEERS,45,35,80,P,2023-11-03
7,21CSL46,PYTHON PROGRAMMING LABORATORY,50,49,99,P,2023-11-03
8,21KSK47,SAMSKRUTIKA KANNADA,39,39,78,P,2023-11-03
9,21UH49,UNIVERSAL HUMAN VALUES AND PROFESSIONAL ETHICS,47,37,84,P,2023-11-03


In [62]:
check.columns = check.iloc[0]
check

,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on
0,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on
1,1MATCS41,"MATHEMATICAL FOUNDATIONS FOR COMPUTING, PROBAB...",47,44,91,P,2023-11-03
2,21CSL481,WEB PROGRAMMING,50,50,100,P,2023-11-03
3,21CS42,DESIGN AND ANALYSIS OF ALGORITHMS,49,40,89,P,2023-11-03
4,21CS43,MICROCONTROLLER AND EMBEDDED SYSTEM,49,40,89,P,2023-11-03
5,21CS44,OPERATING SYSTEM,44,39,83,P,2023-11-03
6,21BE45,BIOLOGY FOR ENGINEERS,45,35,80,P,2023-11-03
7,21CSL46,PYTHON PROGRAMMING LABORATORY,50,49,99,P,2023-11-03
8,21KSK47,SAMSKRUTIKA KANNADA,39,39,78,P,2023-11-03
9,21UH49,UNIVERSAL HUMAN VALUES AND PROFESSIONAL ETHICS,47,37,84,P,2023-11-03


In [64]:
check.drop(0, inplace=True)

In [75]:
check.reset_index(drop=True,inplace=True)
check

,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on
0,1MATCS41,"MATHEMATICAL FOUNDATIONS FOR COMPUTING, PROBAB...",47,44,91,P,2023-11-03
1,21CSL481,WEB PROGRAMMING,50,50,100,P,2023-11-03
2,21CS42,DESIGN AND ANALYSIS OF ALGORITHMS,49,40,89,P,2023-11-03
3,21CS43,MICROCONTROLLER AND EMBEDDED SYSTEM,49,40,89,P,2023-11-03
4,21CS44,OPERATING SYSTEM,44,39,83,P,2023-11-03
5,21BE45,BIOLOGY FOR ENGINEERS,45,35,80,P,2023-11-03
6,21CSL46,PYTHON PROGRAMMING LABORATORY,50,49,99,P,2023-11-03
7,21KSK47,SAMSKRUTIKA KANNADA,39,39,78,P,2023-11-03
8,21UH49,UNIVERSAL HUMAN VALUES AND PROFESSIONAL ETHICS,47,37,84,P,2023-11-03


In [93]:
check.iloc[1:2].reset_index(drop=True)

,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on
0,21CSL481,WEB PROGRAMMING,50,50,100,P,2023-11-03


In [96]:
pd.concat([example,check.iloc[0:1].reset_index(drop=True),check.iloc[1:2].reset_index(drop=True),check.iloc[2:3].reset_index(drop=True)], axis=1)

,USN,Name,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on,Subject Code,...,Total,Result,Announced / Updated on,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on
0,1OX21CS144,SRIKAR V,1MATCS41,"MATHEMATICAL FOUNDATIONS FOR COMPUTING, PROBAB...",47,44,91,P,2023-11-03,21CSL481,...,100,P,2023-11-03,21CS42,DESIGN AND ANALYSIS OF ALGORITHMS,49,40,89,P,2023-11-03


In [101]:
check

,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on
0,1MATCS41,"MATHEMATICAL FOUNDATIONS FOR COMPUTING, PROBAB...",47,44,91,P,2023-11-03
1,21CSL481,WEB PROGRAMMING,50,50,100,P,2023-11-03
2,21CS42,DESIGN AND ANALYSIS OF ALGORITHMS,49,40,89,P,2023-11-03
3,21CS43,MICROCONTROLLER AND EMBEDDED SYSTEM,49,40,89,P,2023-11-03
4,21CS44,OPERATING SYSTEM,44,39,83,P,2023-11-03
5,21BE45,BIOLOGY FOR ENGINEERS,45,35,80,P,2023-11-03
6,21CSL46,PYTHON PROGRAMMING LABORATORY,50,49,99,P,2023-11-03
7,21KSK47,SAMSKRUTIKA KANNADA,39,39,78,P,2023-11-03
8,21UH49,UNIVERSAL HUMAN VALUES AND PROFESSIONAL ETHICS,47,37,84,P,2023-11-03


In [102]:
example

,USN,Name
0,1OX21CS144,SRIKAR V


In [134]:
cols = pd.MultiIndex.from_product([['21MATCS41','21CSL481'], ['INT','EXT','TOT']])
cols

MultiIndex([('21MATCS41', 'INT'),
            ('21MATCS41', 'EXT'),
            ('21MATCS41', 'TOT'),
            ( '21CSL481', 'INT'),
            ( '21CSL481', 'EXT'),
            ( '21CSL481', 'TOT')],
           )

In [122]:
data = [[47,44,91,50,50,100]]

In [139]:
multi = pd.DataFrame(data, columns=cols)
multi

21MATCS41         21CSL481         
        INT EXT TOT      INT EXT  TOT
0        47  44  91       50  50  100

In [144]:
both = pd.concat([example,multi],axis=1)
both

,USN,Name,"(21MATCS41, INT)","(21MATCS41, EXT)","(21MATCS41, TOT)","(21CSL481, INT)","(21CSL481, EXT)","(21CSL481, TOT)"
0,1OX21CS144,SRIKAR V,47,44,91,50,50,100


In [146]:
both = both.set_index(['USN','Name'])
both

,,"(21MATCS41, INT)","(21MATCS41, EXT)","(21MATCS41, TOT)","(21CSL481, INT)","(21CSL481, EXT)","(21CSL481, TOT)"
USN,Name,,,,,,
1OX21CS144,SRIKAR V,47,44,91,50,50,100


In [147]:
both.columns = pd.MultiIndex.from_tuples(both.columns)

In [148]:
both

21MATCS41         21CSL481         
                          INT EXT TOT      INT EXT  TOT
USN        Name                                        
1OX21CS144 SRIKAR V        47  44  91       50  50  100

In [153]:
check

,Subject Code,Subject Name,Internal Marks,External Marks,Total,Result,Announced / Updated on
0,1MATCS41,"MATHEMATICAL FOUNDATIONS FOR COMPUTING, PROBAB...",47,44,91,P,2023-11-03
1,21CSL481,WEB PROGRAMMING,50,50,100,P,2023-11-03
2,21CS42,DESIGN AND ANALYSIS OF ALGORITHMS,49,40,89,P,2023-11-03
3,21CS43,MICROCONTROLLER AND EMBEDDED SYSTEM,49,40,89,P,2023-11-03
4,21CS44,OPERATING SYSTEM,44,39,83,P,2023-11-03
5,21BE45,BIOLOGY FOR ENGINEERS,45,35,80,P,2023-11-03
6,21CSL46,PYTHON PROGRAMMING LABORATORY,50,49,99,P,2023-11-03
7,21KSK47,SAMSKRUTIKA KANNADA,39,39,78,P,2023-11-03
8,21UH49,UNIVERSAL HUMAN VALUES AND PROFESSIONAL ETHICS,47,37,84,P,2023-11-03


In [163]:
values = []
for i in check[['Internal Marks','External Marks','Total']].values:
    for j in i:
        values.append(j)
print(values)

['47', '44', '91', '50', '50', '100', '49', '40', '89', '49', '40', '89', '44', '39', '83', '45', '35', '80', '50', '49', '99', '39', '39', '78', '47', '37', '84']


In [151]:
subcodes = check['Subject Code'].tolist()
subcodes

['1MATCS41',
 '21CSL481',
 '21CS42',
 '21CS43',
 '21CS44',
 '21BE45',
 '21CSL46',
 '21KSK47',
 '21UH49']

In [152]:
marks = ['INT','EXT','TOT']
marks

['INT', 'EXT', 'TOT']

In [164]:
cols = pd.MultiIndex.from_product([subcodes,marks])
cols

MultiIndex([('1MATCS41', 'INT'),
            ('1MATCS41', 'EXT'),
            ('1MATCS41', 'TOT'),
            ('21CSL481', 'INT'),
            ('21CSL481', 'EXT'),
            ('21CSL481', 'TOT'),
            (  '21CS42', 'INT'),
            (  '21CS42', 'EXT'),
            (  '21CS42', 'TOT'),
            (  '21CS43', 'INT'),
            (  '21CS43', 'EXT'),
            (  '21CS43', 'TOT'),
            (  '21CS44', 'INT'),
            (  '21CS44', 'EXT'),
            (  '21CS44', 'TOT'),
            (  '21BE45', 'INT'),
            (  '21BE45', 'EXT'),
            (  '21BE45', 'TOT'),
            ( '21CSL46', 'INT'),
            ( '21CSL46', 'EXT'),
            ( '21CSL46', 'TOT'),
            ( '21KSK47', 'INT'),
            ( '21KSK47', 'EXT'),
            ( '21KSK47', 'TOT'),
            (  '21UH49', 'INT'),
            (  '21UH49', 'EXT'),
            (  '21UH49', 'TOT')],
           )

In [352]:
new = pd.DataFrame(columns=cols)
new

Empty DataFrame
Columns: [(1MATCS41, INT), (1MATCS41, EXT), (1MATCS41, TOT), (21CSL481, INT), (21CSL481, EXT), (21CSL481, TOT), (21CS42, INT), (21CS42, EXT), (21CS42, TOT), (21CS43, INT), (21CS43, EXT), (21CS43, TOT), (21CS44, INT), (21CS44, EXT), (21CS44, TOT), (21BE45, INT), (21BE45, EXT), (21BE45, TOT), (21CSL46, INT), (21CSL46, EXT), (21CSL46, TOT), (21KSK47, INT), (21KSK47, EXT), (21KSK47, TOT), (21UH49, INT), (21UH49, EXT), (21UH49, TOT)]
Index: []

[0 rows x 27 columns]

In [410]:
new.loc[3] = values
new

1MATCS41         21CSL481          21CS42         21CS43  ... 21BE45  \
       INT EXT TOT      INT EXT  TOT    INT EXT TOT    INT  ...    TOT   
0       47  44  91       50  50  100     49  40  89     49  ...     80   
3       47  44  91       50  50  100     49  40  89     49  ...     80   

  21CSL46         21KSK47         21UH49          
      INT EXT TOT     INT EXT TOT    INT EXT TOT  
0      50  49  99      39  39  78     47  37  84  
3      50  49  99      39  39  78     47  37  84  

[2 rows x 27 columns]

In [430]:
new['Total'] = 200
new

1MATCS41         21CSL481          21CS42         21CS43  ... 21CSL46      \
       INT EXT TOT      INT EXT  TOT    INT EXT TOT    INT  ...     INT EXT   
0       47  44  91       50  50  100     49  40  89     49  ...      50  49   
3       47  44  91       50  50  100     49  40  89     49  ...      50  49   

      21KSK47         21UH49         Total  
  TOT     INT EXT TOT    INT EXT TOT        
0  99      39  39  78     47  37  84   200  
3  99      39  39  78     47  37  84   200  

[2 rows x 28 columns]

In [438]:
new.at[0, 'Total'] = 250
new

1MATCS41         21CSL481          21CS42         21CS43  ... 21CSL46      \
       INT EXT TOT      INT EXT  TOT    INT EXT TOT    INT  ...     INT EXT   
0       47  44  91       50  50  100     49  40  89     49  ...      50  49   
3       47  44  91       50  50  100     49  40  89     49  ...      50  49   

      21KSK47         21UH49         Total  
  TOT     INT EXT TOT    INT EXT TOT        
0  99      39  39  78     47  37  84   250  
3  99      39  39  78     47  37  84   200  

[2 rows x 28 columns]

In [341]:
new.drop(['Total'],axis=1,inplace=True)

/var/folders/2j/l_xbmzz11mngr8tgsh_ww_tr0000gn/T/ipykernel_24231/1255557689.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new.drop(['Total'],axis=1,inplace=True)


In [343]:
new.loc[1] = values
new

1MATCS41         21CSL481          21CS42         21CS43  ... 21BE45  \
       INT EXT TOT      INT EXT  TOT    INT EXT TOT    INT  ...    TOT   
0       47  44  91       50  50  100     49  40  89     49  ...     80   
1       47  44  91       50  50  100     49  40  89     49  ...     80   

  21CSL46         21KSK47         21UH49          
      INT EXT TOT     INT EXT TOT    INT EXT TOT  
0      50  49  99      39  39  78     47  37  84  
1      50  49  99      39  39  78     47  37  84  

[2 rows x 27 columns]

In [180]:
new.loc[0]['21CSL481']['TOT']

100

In [176]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   (1MATCS41, INT)  1 non-null      object
 1   (1MATCS41, EXT)  1 non-null      object
 2   (1MATCS41, TOT)  1 non-null      object
 3   (21CSL481, INT)  1 non-null      object
 4   (21CSL481, EXT)  1 non-null      object
 5   (21CSL481, TOT)  1 non-null      object
 6   (21CS42, INT)    1 non-null      object
 7   (21CS42, EXT)    1 non-null      object
 8   (21CS42, TOT)    1 non-null      object
 9   (21CS43, INT)    1 non-null      object
 10  (21CS43, EXT)    1 non-null      object
 11  (21CS43, TOT)    1 non-null      object
 12  (21CS44, INT)    1 non-null      object
 13  (21CS44, EXT)    1 non-null      object
 14  (21CS44, TOT)    1 non-null      object
 15  (21BE45, INT)    1 non-null      object
 16  (21BE45, EXT)    1 non-null      object
 17  (21BE45, TOT)    1 non-null      object

In [178]:
new = new.astype(int)

In [179]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   (1MATCS41, INT)  1 non-null      int64
 1   (1MATCS41, EXT)  1 non-null      int64
 2   (1MATCS41, TOT)  1 non-null      int64
 3   (21CSL481, INT)  1 non-null      int64
 4   (21CSL481, EXT)  1 non-null      int64
 5   (21CSL481, TOT)  1 non-null      int64
 6   (21CS42, INT)    1 non-null      int64
 7   (21CS42, EXT)    1 non-null      int64
 8   (21CS42, TOT)    1 non-null      int64
 9   (21CS43, INT)    1 non-null      int64
 10  (21CS43, EXT)    1 non-null      int64
 11  (21CS43, TOT)    1 non-null      int64
 12  (21CS44, INT)    1 non-null      int64
 13  (21CS44, EXT)    1 non-null      int64
 14  (21CS44, TOT)    1 non-null      int64
 15  (21BE45, INT)    1 non-null      int64
 16  (21BE45, EXT)    1 non-null      int64
 17  (21BE45, TOT)    1 non-null      int64
 18  (21CSL46, INT)

In [181]:
new['Total'] = 155

In [185]:
new.loc[0]['Total'] = 200

In [187]:
new

1MATCS41         21CSL481          21CS42         21CS43  ... 21CSL46      \
       INT EXT TOT      INT EXT  TOT    INT EXT TOT    INT  ...     INT EXT   
0       47  44  91       50  50  100     49  40  89     49  ...      50  49   

      21KSK47         21UH49         Total  
  TOT     INT EXT TOT    INT EXT TOT        
0  99      39  39  78     47  37  84   200  

[1 rows x 28 columns]

## Drive access and download

In [188]:
drive

In [223]:
response = (
          drive.files()
          .list(
              q="name contains 'Class 4c'",
              spaces="drive",
              fields="nextPageToken, files(id, name)",).execute())

In [224]:
response

{'files': [{'id': '1MAWB8k27Xw_elRJTvaQ8LY0UGzuQZxod', 'name': 'Class 4C'}]}

In [225]:
response.get("files", [])

[{'id': '1MAWB8k27Xw_elRJTvaQ8LY0UGzuQZxod', 'name': 'Class 4C'}]

In [299]:
folder_id = "https://drive.google.com/drive/u/0/folders/1MAWB8k27Xw_elRJTvaQ8LY0UGzuQZxod"

In [300]:
folder_id = folder_id.split("/")[-1:][0]
folder_id

'1MAWB8k27Xw_elRJTvaQ8LY0UGzuQZxod'

In [302]:
results = drive.files().list(q=f"'{folder_id}' in parents",pageSize=5, orderBy='name', fields="files(id, name)").execute()

In [309]:
id = results.get("files")[0]['id']

In [304]:
results

KeyError: 0

In [310]:
pdf = drive.files().get_media(fileId = id)

In [311]:
import io

In [312]:
pdf_content = io.BytesIO()

In [313]:
downloader = MediaIoBaseDownload(pdf_content, pdf)

In [314]:
done = False
while not done:
    status,done = downloader.next_chunk()
pdf_content.seek(0)

0

In [315]:
pdf_content

In [316]:
pdf_doc = fitz.open(stream=pdf_content)
pdf_doc

Document('', <memory, doc# 3>)

In [317]:
page = pdf_doc[0] # get the 1st page of the document
tabs = page.find_tables()

In [320]:
tabs[0].extract()

[['University Seat Number', ': 1OX21CS116'], ['Student Name', ': ROHAN VATS']]

## Function Call

In [874]:
def files_list(folder_link):
    drive = authenticate_driveAPI(SCOPES=SCOPES)
    folder_id = folder_link.split("/")[-1:][0]
    print("Fetching list of files...")
    try:
        response = drive.files().list(
                q=f"'{folder_id}' in parents",
                orderBy='name',
                fields="files(id, name)").execute()
        files_list = response.get("files", [])
        if not files_list:
            print("Warning: No Files in the Folder...")
        else:
            print("Files (id,name) fetched succesfully!!")
            return files_list
    except HttpError as e:
        if e.resp.status == 404:
            print("Error 404: Folder not found...")
        else:
            print(f"Error: {e}")

In [881]:
folder_id = "https://drive.google.com/drive/u/0/folders/1eqcKusmc-a871YwI0CSYGZoUionk0nSn"

In [882]:
files_list(folder_id)

Fetching list of files...
Files (id,name) fetched succesfully!!


[{'id': '1gI7yxyDUaKi8HKiNB7IArHUeyTTKR8pO', 'name': '1OX21CS116.pdf'},
 {'id': '1VFQwpPvRhuH8WBw8_nXTD9Ga4RhJsrpL', 'name': '1OX21CS117.pdf'},
 {'id': '1um42Eumkmy_rLMlXEnCC85T-NUkbsHYJ', 'name': '1OX21CS118.pdf'},
 {'id': '1-ZzkU9uAuZWNxJINlNskoh9UoYySnFQI', 'name': '1OX21CS119.pdf'},
 {'id': '1XGkMCffXKqOpA5mcK-89HgznOc9yo8vX', 'name': '1OX21CS120.pdf'},
 {'id': '1xWrX1P3ZAUVNQHPXfW_h2Ch9KTIwCvyJ', 'name': '1OX21CS121.pdf'},
 {'id': '1ei0E6vFoZSx14wu3eCV_CBuCYGZ_MFXV', 'name': '1OX21CS122.pdf'},
 {'id': '1NtAptikWbLnd1EeTlWcpbtkYAxTeJdb1', 'name': '1OX21CS123.pdf'},
 {'id': '1qLhF6BD7n4PPn-aOTPgzryCR4aL7stnU', 'name': '1OX21CS124.pdf'},
 {'id': '1JLyWmf_PZJFSholhM6Z06JA7sdDyTjRo', 'name': '1OX21CS125.pdf'},
 {'id': '1c93NsDf-NjNh-aMe9EOuy66nvAwZi8-V', 'name': '1OX21CS126.pdf'},
 {'id': '12UBqvKshNVs_HHvnq3CiCWO6xCzM9ZNE', 'name': '1OX21CS127.pdf'},
 {'id': '10b7R8kWwtgqw6K2U5AbeaKA-hUmyjqZK', 'name': '1OX21CS128.pdf'},
 {'id': '1E8XdWzCwJdCMtTfROidcClFxGrR9T_q2', 'name': '1OX21CS129

In [326]:
id = files_list(folder_id,5)[0]

Fetching list of files...
Files (id,name) fetched succesfully!!


In [327]:
id

{'id': '1gI7yxyDUaKi8HKiNB7IArHUeyTTKR8pO', 'name': '1OX21CS116.pdf'}

In [511]:
def extract_pdf_content(file):
    drive = authenticate_driveAPI(SCOPES=SCOPES)
    pdf_content = io.BytesIO()
    try:
        pdf = drive.files().get_media(fileId = file['id'])
        downloader = MediaIoBaseDownload(pdf_content,pdf)
        done = False
        while not done:
            status,done = downloader.next_chunk()
            sys.stdout.write(f"Extracting pdf {file['name']}...    Extracted 0%")
            for i in range(10):
                time.sleep(0.1)
                dash = '-'
                sys.stdout.write(f'{dash}')
                sys.stdout.flush()
            if done:
                sys.stdout.write(f"{int(status.progress() * 100)}%\n")
                sys.stdout.flush()
        pdf_content.seek(0)
        pdf_doc = fitz.open(stream=pdf_content)
        page = pdf_doc[0]
        tables = page.find_tables()
        details = tables[0].extract()
        details = [details[0][1][2:],details[1][1][2:]]
        marks = tables[1].extract()
        marks_df = pd.DataFrame(marks)
        marks_df = marks_df.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x)
        marks_df.columns = marks_df.iloc[0]
        marks_df.drop(0, inplace=True)
        marks_df.reset_index(drop=True,inplace=True)
        marks_df.drop(['Subject Name','Announced / Updated on'],axis=1,inplace=True)
        return details,marks_df
    except HttpError as e:
        if e.resp.status == 404:
            print("Error: File not found...")
        elif e.resp.status == 403:
            print("Error: Permission denied. You do not have access to this file...")
        else:
            print(f"Error: {e}")

In [512]:
extract_pdf_content(id)

Extracting pdf 1OX21CS116.pdf...    Extracted 0%----------100%


(['1OX21CS116', 'ROHAN VATS'],
 0 Subject Code Internal Marks External Marks Total Result
 0    21MATCS41             46             12    58      F
 1     21CSL481             50             50   100      P
 2       21CS42             47             22    69      P
 3       21CS43             43             22    65      P
 4       21CS44             38             14    52      F
 5       21BE45             37             19    56      P
 6      21CSL46             50             50   100      P
 7      21KBK47             49             45    94      P
 8       21UH49             50             36    86      P
 9      21INT49             99              0    99      P)

In [884]:
def extract_process_student_result():
    folder_link = input("Please enter the drive link: ")
    files = files_list(folder_link)
    details_df = pd.DataFrame(columns=['USN','Name'])
    students_marks = pd.DataFrame()
    if files:
        for i in tqdm(range(len(files))):
            details, marks_df = extract_pdf_content(file=files[i])
            if len(marks_df) == 1 or len(marks_df) == 0:
                print(f"Ignoring {details[0]} because He/She has dropped out")
                continue
            details_df.loc[i] = details
            if i == 0:
                subcodes = marks_df['Subject Code'].tolist()
                marks = ['INT','EXT','TOT','RESULT']
                cols = pd.MultiIndex.from_product([subcodes,marks])
                students_marks = pd.DataFrame(columns=cols)
                students_marks['Total'] = 0
                students_marks['Percentage'] = 0
                students_marks['SGPA'] = 0
                credits = []
                print("Please enter the credits:")
                for code in subcodes:
                    c = int(input(f"{code}:"))
                    credits.append(c)
            values = []
            for mark in marks_df[['Internal Marks','External Marks','Total','Result']].values:
                for val in mark:
                    values.append(val)
            values.append('0')
            values.append('0')
            values.append('0')
            students_marks.loc[i] = values
            total = 0
            totals = []
            for code in subcodes:
                totals.append(int(students_marks.loc[i][code]['TOT']))
                total += int(students_marks.loc[i][code]['TOT'])
            students_marks.at[i, 'Total'] = total
            students_marks.at[i, 'Percentage'] = total / len(subcodes)
            students_marks.at[i, 'SGPA'] = compute_SGPA(totals,credits)
        students_marks = pd.concat([details_df,students_marks],axis=1)
        students_marks = students_marks.set_index(['USN','Name'])
        students_marks.columns = pd.MultiIndex.from_tuples(students_marks.columns)
        print("Extraction Succesful!!")
        return students_marks
    else:
        print("Warning: Folder empty or folder not found...")

In [520]:
def compute_SGPA(totals,credits):
    sgpa = 0
    sum = 0
    for total,credit in zip(totals,credits):
        sum += credit
        if total >= 90:
            sgpa += (10*credit)
        elif total >=80:
            sgpa += (9*credit)
        elif total >= 70:
            sgpa += (8*credit)
        elif total >= 60:
            sgpa += (7*credit)
        elif total >= 50:
            sgpa += (6*credit)
        elif total >= 40:
            sgpa += (5*credit)
        else:
            sgpa += 0
    sgpa = sgpa / sum
    return sgpa

In [593]:
extract_process_student_result()

Please enter the drive link:  https://drive.google.com/drive/u/0/folders/1MAWB8k27Xw_elRJTvaQ8LY0UGzuQZxod
Fetching list of files...
Files (id,name) fetched succesfully!!


  0%|          | 0/4 [00:00<?, ?it/s]

Extracting pdf 1OX21CS116.pdf...    Extracted 0%----------100%
Please enter the credits:
21MATCS41: 3
21CSL481: 1
21CS42: 4
21CS43: 4
21CS44: 3
21BE45: 2
21CSL46: 1
21KBK47: 1
21UH49: 1
21INT49: 2
Extracting pdf 1OX21CS117.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS118.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS119.pdf...    Extracted 0%----------100%
Extraction Succesful!!


21MATCS41                 21CSL481           \
                                       INT EXT  TOT RESULT      INT EXT  TOT   
USN        Name                                                                
1OX21CS116 ROHAN VATS                   46  12   58      F       50  50  100   
1OX21CS117 ROJA MK                      49  43   92      P       49  47   96   
1OX21CS118 ROOPA R                      50  50  100      P       50  34   84   
1OX21CS119 RUSHIKESH B KATTIMANI        39  32   71      P       50  46   96   

                                        21CS42      ... 21UH49             \
                                 RESULT    INT EXT  ...    EXT TOT RESULT   
USN        Name                                     ...                     
1OX21CS116 ROHAN VATS                 P     47  22  ...     36  86      P   
1OX21CS117 ROJA MK                    P     45  25  ...     41  91      P   
1OX21CS118 ROOPA R                    P     46  31  ...     41  91      P   
1OX21CS119 RUSHIKESH B KATTIMANI      P     37  31  ...     32  81      P   

                                 21INT49                Total Percentage  \
                                     INT EXT TOT RESULT                    
USN        Name                                                            
1OX21CS116 ROHAN VATS                 99   0  99      P   779       77.9   
1OX21CS117 ROJA MK                    95   0  95      P   870       87.0   
1OX21CS118 ROOPA R                    97   0  97      P   892       89.2   
1OX21CS119 RUSHIKESH B KATTIMANI      97   0  97      P   819       81.9   

                                      SGPA  
                                            
USN        Name                             
1OX21CS116 ROHAN VATS             7.409091  
1OX21CS117 ROJA MK                9.090909  
1OX21CS118 ROOPA R                9.136364  
1OX21CS119 RUSHIKESH B KATTIMANI  8.136364  

[4 rows x 43 columns]

In [521]:
students_marks = preprocess_content()

Please enter the drive link:  https://drive.google.com/drive/u/0/folders/1MAWB8k27Xw_elRJTvaQ8LY0UGzuQZxod
Fetching list of files...
Files (id,name) fetched succesfully!!


  0%|          | 0/40 [00:00<?, ?it/s]

Extracting pdf 1OX21CS116.pdf...    Extracted 0%----------100%
Please enter the credits:
21MATCS41: 3
21CSL481: 1
21CS42: 4
21CS43: 4
21CS44: 3
21BE45: 2
21CSL46: 1
21KBK47: 1
21UH49: 1
21INT49: 2
Extracting pdf 1OX21CS117.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS118.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS119.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS120.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS121.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS122.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS123.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS124.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS125.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS126.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS127.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS128.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS129.pdf...    Extracted 0

In [522]:
students_marks

21MATCS41                 21CSL481  \
                                                INT EXT  TOT RESULT      INT   
USN        Name                                                                
1OX21CS116 ROHAN VATS                            46  12   58      F       50   
1OX21CS117 ROJA MK                               49  43   92      P       49   
1OX21CS118 ROOPA R                               50  50  100      P       50   
1OX21CS119 RUSHIKESH B KATTIMANI                 39  32   71      P       50   
1OX21CS120 S SARVESHWAR                          49  43   92      P       50   
1OX21CS121 SABAH TABASUM                         50  42   92      P       50   
1OX21CS122 SAGAR SHEKHARAGOUDA CHINTAMANI        43  32   75      P       50   
1OX21CS123 SAHANA R                              50  36   86      P       50   
1OX21CS124 SAHEB KAMILA                          42  25   67      P       50   
1OX21CS125 SAI THILAK T                          26  11   37      F       49   
1OX21CS126 SAMIKSHA SIMMI                        26   7   33      F       22   
1OX21CS127 SAMPRIT BERA                          26   0   26      F       22   
1OX21CS128 SANDHYA S                             45  39   84      P       50   
1OX21CS129 SANIDHYA PUZARI                       26   2   28      F       37   
1OX21CS130 SANJAY SAGAR M                        37  29   66      P       48   
1OX21CS131 SANTHOSH R                            47  47   94      P       39   
1OX21CS132 SATHISH N R                           27   6   33      F       30   
1OX21CS133 SHANKAR                               29   5   34      F       38   
1OX21CS134 SHANTHINI R                           50  48   98      P       46   
1OX21CS135 SHARAN I G                            37  27   64      P       49   
1OX21CS136 SHASHANK MURTHY K                     31  26   57      P       50   
1OX21CS137 SHASHANTH M                           39  28   67      P       50   
1OX21CS138 SHASHANTH R                           50  42   92      P       50   
1OX21CS139 SHEELA M                              45  47   92      P       49   
1OX21CS140 SHINDU                                41  33   74      P       49   
1OX21CS141 SHIVKANT                              36  30   66      P       44   
1OX21CS142 SHREYAS BYAHATTI                      39  38   77      P       49   
1OX21CS143 SNEHA S                               33  22   55      P       45   
1OX21CS144 SRIKAR V                              47  44   91      P       50   
1OX21CS145 SRIMATHI J                            46  46   92      P       50   
1OX21CS146 SRINIVAS N                            42  23   65      P       50   
1OX21CS147 SUKSHITHA S                           46  31   77      P       50   
1OX21CS148 SUMON DAS                             32  26   58      P       50   
1OX21CS149 SUPRIYA                               42  27   69      P       50   
1OX21CS150 SUSHMA                                47  50   97      P       50   
1OX21CS152 SYED IBRAHIM                          45  46   91      P       50   
1OX21CS153 SYED MOHAMMED WAHI                    25   4   29      F       22   
1OX21CS154 TEJASWINI J                           45  32   77      P       50   
1OX21CS155 TEPPALA SNEHA                         50  42   92      P       50   

                                                          21CS42      ...  \
                                          EXT  TOT RESULT    INT EXT  ...   
USN        Name                                                       ...   
1OX21CS116 ROHAN VATS                      50  100      P     47  22  ...   
1OX21CS117 ROJA MK                         47   96      P     45  25  ...   
1OX21CS118 ROOPA R                         34   84      P     46  31  ...   
1OX21CS119 RUSHIKESH B KATTIMANI           46   96      P     37  31  ...   
1OX21CS120 S SARVESHWAR                    50  100      P     49  38  ...   
1OX21CS121 SABAH TABASUM                   50  100      P     50 

In [539]:
pd.DataFrame(students_marks[('21MATCS41', 'RESULT')].value_counts())

,21MATCS41
,RESULT
P,31
F,8


In [540]:
pfa = pd.DataFrame(students_marks[('21MATCS41', 'RESULT')].value_counts())
pfa

,21MATCS41
,RESULT
P,31
F,8


In [541]:
pfa = pd.concat([pfa,students_marks[('21CSL481', 'RESULT')].value_counts()],axis=1)
pfa

,21MATCS41,21CSL481
,RESULT,RESULT
P,31.0,36.0
F,8.0,NaN
A,NaN,3.0


In [546]:
x = students_marks.columns[3]

'21MATCS41'

In [548]:
students_marks[x]

USN         Name                          
1OX21CS116  ROHAN VATS                        F
1OX21CS117  ROJA MK                           P
1OX21CS118  ROOPA R                           P
1OX21CS119  RUSHIKESH B KATTIMANI             P
1OX21CS120  S SARVESHWAR                      P
1OX21CS121  SABAH TABASUM                     P
1OX21CS122  SAGAR SHEKHARAGOUDA CHINTAMANI    P
1OX21CS123  SAHANA R                          P
1OX21CS124  SAHEB KAMILA                      P
1OX21CS125  SAI THILAK T                      F
1OX21CS126  SAMIKSHA SIMMI                    F
1OX21CS127  SAMPRIT BERA                      F
1OX21CS128  SANDHYA S                         P
1OX21CS129  SANIDHYA PUZARI                   F
1OX21CS130  SANJAY SAGAR M                    P
1OX21CS131  SANTHOSH R                        P
1OX21CS132  SATHISH N R                       F
1OX21CS133  SHANKAR                           F
1OX21CS134  SHANTHINI R                       P
1OX21CS135  SHARAN I G                       

In [551]:
pfa = pd.DataFrame()
for i in range(3,41,4):
    col = students_marks.columns[i]
    if i == 3:
        for 
        pfa = pd.DataFrame(students_marks[col].value_counts())
    else:
        pfa = pd.concat([pfa,students_marks[col].value_counts()],axis=1)
pfa

,21MATCS41,21CSL481,21CS42,21CS43,21CS44,21BE45,21CSL46,21KBK47,21UH49,21INT49
,RESULT,RESULT,RESULT,RESULT,RESULT,RESULT,RESULT,RESULT,RESULT,RESULT
P,31.0,36.0,35.0,33,34,37.0,36.0,39.0,39.0,39.0
F,8.0,NaN,4.0,3,3,NaN,NaN,NaN,NaN,NaN
A,NaN,3.0,NaN,3,2,2.0,3.0,NaN,NaN,NaN


In [559]:
z = pd.DataFrame([[1],[2],[3]],columns=['ZZ'],index=['P','F','A'])
z

,ZZ
P,1
F,2
A,3


In [563]:
pd.concat([pd.Series([3,4,5],index=['P','F','A']),z],axis=1)

,0,ZZ
P,3,1
F,4,2
A,5,3


In [554]:
for i in range(3,41,4):
    col = students_marks.columns[i]
    for c in students_marks[col]:
        print(c)

F
P
P
P
P
P
P
P
P
F
F
F
P
F
P
P
F
F
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
F
P
P
P
P
P
P
P
P
P
P
P
P
A
A
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
A
P
P
P
P
P
P
P
P
P
P
P
P
P
F
P
F
P
P
F
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
F
P
P
P
P
P
P
P
P
P
P
P
P
F
A
P
P
P
F
P
A
P
P
P
P
P
P
P
F
P
P
P
P
P
P
P
P
P
P
A
P
P
F
P
P
P
P
P
P
P
P
P
F
A
P
F
P
P
P
A
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
A
P
P
P
P
P
A
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
A
A
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
A
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P
P


In [538]:
students_marks['21MATCS41']['RESULT']

USN         Name                          
1OX21CS116  ROHAN VATS                        F
1OX21CS117  ROJA MK                           P
1OX21CS118  ROOPA R                           P
1OX21CS119  RUSHIKESH B KATTIMANI             P
1OX21CS120  S SARVESHWAR                      P
1OX21CS121  SABAH TABASUM                     P
1OX21CS122  SAGAR SHEKHARAGOUDA CHINTAMANI    P
1OX21CS123  SAHANA R                          P
1OX21CS124  SAHEB KAMILA                      P
1OX21CS125  SAI THILAK T                      F
1OX21CS126  SAMIKSHA SIMMI                    F
1OX21CS127  SAMPRIT BERA                      F
1OX21CS128  SANDHYA S                         P
1OX21CS129  SANIDHYA PUZARI                   F
1OX21CS130  SANJAY SAGAR M                    P
1OX21CS131  SANTHOSH R                        P
1OX21CS132  SATHISH N R                       F
1OX21CS133  SHANKAR                           F
1OX21CS134  SHANTHINI R                       P
1OX21CS135  SHARAN I G                       

In [523]:
top10 = students_marks.sort_values(by=['Percentage','SGPA'], ascending=False)[:10][['Total','Percentage','SGPA']]
top10

,,Total,Percentage,SGPA
,,,,
USN,Name,,,
1OX21CS123,SAHANA R,923,92.3,9.363636
1OX21CS145,SRIMATHI J,915,91.5,9.545455
1OX21CS152,SYED IBRAHIM,913,91.3,9.5
1OX21CS121,SABAH TABASUM,913,91.3,9.272727
1OX21CS155,TEPPALA SNEHA,911,91.1,9.545455
1OX21CS120,S SARVESHWAR,897,89.7,9.363636
1OX21CS138,SHASHANTH R,896,89.6,9.227273
1OX21CS128,SANDHYA S,894,89.4,9.227273


In [510]:
students_marks.sort_values(by=['Percentage','SGPA'], ascending=True)[:10][['Total','Percentage','SGPA']]

,,Total,Percentage,SGPA
,,,,
USN,Name,,,
1OX21CS127,SAMPRIT BERA,306,30.6,1.0
1OX21CS168,YASHAS ADITYA L,335,33.5,1.5
1OX21CS126,SAMIKSHA SIMMI,434,43.4,2.818182
1OX21CS153,SYED MOHAMMED WAHI,463,46.3,3.772727
1OX21CS158,UJJWAL M K,484,48.4,3.227273
1OX21CS132,SATHISH N R,508,50.8,4.181818
1OX21CS133,SHANKAR,571,57.1,3.5
1OX21CS129,SANIDHYA PUZARI,630,63.0,5.818182


In [668]:
def compute_grades(students_marks):
    index = ['FCD','FC','SC','Fail','Absent','Total']
    sys.stdout.write('Computing grades...')
    pfa = pd.DataFrame()
    cols = []
    for i in tqdm(range(3,len(students_marks.columns)-2,4)):
        result = students_marks.columns[i]
        total = students_marks.columns[i-1]
        fcd = 0
        fc = 0
        sc = 0
        f = 0
        a = 0
        for res,tot in zip(students_marks[result],students_marks[total]):
            if res == 'P':
                if int(tot) >= 75:
                    fcd += 1
                elif int(tot) >= 60:
                    fc += 1
                else:
                    sc += 1
            elif res == 'F':
                f += 1
            elif res == 'A':
                a += 1
        count = len(students_marks[result])
        if i == 3:
            pfa = pd.DataFrame([[fcd],[fc],[sc],[f],[a],[count]],columns=[result[0]],index=index)
        else:
            pfa = pd.concat([pfa,pd.DataFrame([fcd,fc,sc,f,a,count],index=index,columns=[result[0]])],axis = 1)
        cols.append(result)
    students_marks.drop(cols,axis=1,inplace=True)
    pfa['Total'] = 0
    pfa['Percentage'] = 0.0
    for index in pfa.index.tolist():
        nums = pfa.loc[index].tolist()[:-1]
        pfa.at[index, 'Total'] = sum(nums)
    for index in pfa.index.tolist():
        if index == 'Total':
            pfa.at[index, 'Percentage'] = 100.0
        else:
            total = pfa['Total'].loc[index]
            pfa.at[index, 'Percentage'] = (total / pfa['Total'].loc['Total'])*100
    sys.stdout.write('Completed!!')
    return pfa

In [654]:
grades = compute_grades(students_marks)

Computing grades...

  0%|          | 0/10 [00:00<?, ?it/s]

Completed!!

In [655]:
grades

,21MATCS41,21CSL481,21CS42,21CS43,21CS44,21BE45,21CSL46,21KBK47,21UH49,21INT49,Total,Percentage
FCD,19,34,15,18,20,21,35,33,31,37,263,67.435897
FC,9,2,14,12,11,9,1,3,3,1,65,16.666667
SC,3,0,6,3,3,7,0,3,5,1,31,7.948718
Fail,8,0,4,3,3,0,0,0,0,0,18,4.615385
Absent,0,3,0,3,2,2,3,0,0,0,13,3.333333
Total,39,39,39,39,39,39,39,39,39,39,390,100.000000


In [898]:
def generate_student_result():
    students_marks = extract_process_student_result()
    top10 = students_marks.sort_values(by=['Percentage','SGPA'], ascending=False)[:10][['Total','Percentage','SGPA']]
    grades = compute_grades(students_marks=students_marks)
    students_marks.reset_index(inplace=True)
    top10.reset_index(inplace=True)
    grades.reset_index(inplace=True)
    sheets = authenticate_sheetsAPI(SCOPES=SCOPES)
    spreadsheetId = create_new_spreadsheet(service=sheets)
    change_access_permission(spreadsheetId=spreadsheetId)
    start_rows = [2, len(students_marks) + 5, len(students_marks) + len(top10) + 10]
    dataframes = [students_marks, top10, grades]
    write_to_sheet(sheets, dataframes, start_rows, spreadsheetId)
    spreadsheet_link = f"https://docs.google.com/spreadsheets/d/{spreadsheetId}"
    print(f"Access it here: {spreadsheet_link}")
    print("Do you want to recieve the link through mail?")
    mail = int(input("Yes: 1? or No: 0?"))
    if mail == 1:
        receiver_email = input("Please enter your email id:")
        send_email(receiver_email,spreadsheet_link)
        print("Thank you!")
    else:
        print("Thank you!")

In [901]:
generate_student_result()

Please enter the drive link:  https://drive.google.com/drive/folders/1cTHAqpz3RzeqqTBhziEILA6uWUkkAfV1
Fetching list of files...
Files (id,name) fetched succesfully!!


  0%|          | 0/56 [00:00<?, ?it/s]

Extracting pdf 1OX21CS116.pdf...    Extracted 0%----------100%
Please enter the credits:
21MATCS41: 3
21CSL481: 1
21CS42: 4
21CS43: 4
21CS44: 3
21BE45: 2
21CSL46: 1
21KBK47: 1
21UH49: 1
21INT49: 2
Extracting pdf 1OX21CS117.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS118.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS119.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS120.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS121.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS122.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS123.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS124.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS125.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS126.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS127.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS128.pdf...    Extracted 0%----------100%
Extracting pdf 1OX21CS129.pdf...    Extracted 0

  0%|          | 0/10 [00:00<?, ?it/s]

Completed!!Please enter the tile of new spreadsheet: Result Extractor Check
Succesfully created spreadsheet - Result Extractor Check
Spreadsheet permissions updated: anyone with link can edit...
Writing dataframes to spreadsheet...


  0%|          | 0/3 [00:00<?, ?it/s]

Writing Completed!!
Access it here: https://docs.google.com/spreadsheets/d/1nqa_D65Udk3kM5Rw5mji2s8j5nx3N4XUNQeQA1wwoqk
Do you want to recieve the link through mail?
Yes: 1? or No: 0? 1
Please enter your email id: vsrikar44@gmail.com
Preparing to send mail...
Connecting to server...
Email sent successfully!!
Thank you!


In [702]:
def create_new_spreadsheet(service):
    try:
        sheetName = input("Please enter the tile of new spreadsheet:")
        spreadsheet = {
            'properties' : {'title' : sheetName}
        }
        spreadsheet = service.spreadsheets().create(body=spreadsheet, fields='spreadsheetId').execute()
        print(f"Succesfully created spreadsheet - {sheetName}")
        return spreadsheet['spreadsheetId']
    except HttpError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Unexpected Error: {e}")

In [695]:
def change_access_permission(spreadsheetId):
    service = authenticate_driveAPI(SCOPES=SCOPES)
    try:
        permissions = {
            'type' : 'anyone',
            'role' : 'writer'
        }
        service.permissions().create(fileId=spreadsheetId, body=permissions).execute()
        print("Spreadsheet permissions updated: anyone with link can edit...")
    except HttpError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Unexpected Error: {e}")

In [873]:
def write_to_sheet(service, dataframes, start_rows, spreadsheetId):
    try:
        print("Writing dataframes to spreadsheet...")
        for i in tqdm(range(len(dataframes))):
            data = dataframes[i].values.tolist()
            if i == 1:
                dataframes[i].columns = [''.join(map(str, col)) for col in dataframes[i].columns]
            cols = []
            subs = []
            if i == 0:
                
                columnss = dataframes[i].columns.tolist()
                for j in range(len(columnss)):
                    if j < 2 or j > (len(columnss)-4):
                        cols.append(columnss[j][0])
                    else:
                        cols.append(columnss[j][1])
                        subs.append(columnss[j][0])
                body = {"values" : [subs]}
                service.spreadsheets().values().update(spreadsheetId=spreadsheetId,
                                                  range=f"C1",
                                                  valueInputOption="RAW",
                                                  body=body).execute()
            if i != 0:
                headers = [dataframes[i].columns.tolist()]
            else:
                headers = [cols]
            body = {"values" : headers}
            service.spreadsheets().values().update(spreadsheetId=spreadsheetId,
                                                  range=f"A{start_rows[i]}",
                                                  valueInputOption="RAW",
                                                  body=body).execute()
            body = {"values" : data}
            service.spreadsheets().values().update(spreadsheetId=spreadsheetId,
                                                  range=f"A{start_rows[i]+1}",
                                                  valueInputOption="RAW",
                                                  body=body).execute()
        print("Writing Completed!!")
    except HttpError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Unexpected Error: {e}")

In [902]:
def send_email(receiver_email,spreadsheetLink):
    print("Preparing to send mail...")
    sender_email = "srikarvuchiha@gmail.com"
    password = "glhy xvwc uaxs rstn"
    message = EmailMessage()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = "VTU Extracted Results Link"
    message.set_content(f"""Hey,
        
    Here is the spreadsheets with the VTU result extracted. You can access the sheet by clicking on the link below.
    Access it here: {sheetLink}
        
    Thanking You,
    Srikar V""")
    
    print("Connecting to server...")
    # Connect to the SMTP server
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, password)
        # Send email
        server.send_message(message)

    print("Email sent successfully!!")

In [858]:
top10.reset_index().columns

MultiIndex([(       'USN', ''),
            (      'Name', ''),
            (     'Total', ''),
            ('Percentage', ''),
            (      'SGPA', '')],
           )

In [843]:
check = students_marks.copy()
check

21MATCS41                 21CSL481  \
                                                INT EXT  TOT RESULT      INT   
USN        Name                                                                
1OX21CS116 ROHAN VATS                            46  12   58      F       50   
1OX21CS117 ROJA MK                               49  43   92      P       49   
1OX21CS118 ROOPA R                               50  50  100      P       50   
1OX21CS119 RUSHIKESH B KATTIMANI                 39  32   71      P       50   
1OX21CS120 S SARVESHWAR                          49  43   92      P       50   
1OX21CS121 SABAH TABASUM                         50  42   92      P       50   
1OX21CS122 SAGAR SHEKHARAGOUDA CHINTAMANI        43  32   75      P       50   
1OX21CS123 SAHANA R                              50  36   86      P       50   
1OX21CS124 SAHEB KAMILA                          42  25   67      P       50   
1OX21CS125 SAI THILAK T                          26  11   37      F       49   
1OX21CS126 SAMIKSHA SIMMI                        26   7   33      F       22   
1OX21CS127 SAMPRIT BERA                          26   0   26      F       22   
1OX21CS128 SANDHYA S                             45  39   84      P       50   
1OX21CS129 SANIDHYA PUZARI                       26   2   28      F       37   
1OX21CS130 SANJAY SAGAR M                        37  29   66      P       48   
1OX21CS131 SANTHOSH R                            47  47   94      P       39   
1OX21CS132 SATHISH N R                           27   6   33      F       30   
1OX21CS133 SHANKAR                               29   5   34      F       38   
1OX21CS134 SHANTHINI R                           50  48   98      P       46   
1OX21CS135 SHARAN I G                            37  27   64      P       49   
1OX21CS136 SHASHANK MURTHY K                     31  26   57      P       50   
1OX21CS137 SHASHANTH M                           39  28   67      P       50   
1OX21CS138 SHASHANTH R                           50  42   92      P       50   
1OX21CS139 SHEELA M                              45  47   92      P       49   
1OX21CS140 SHINDU                                41  33   74      P       49   
1OX21CS141 SHIVKANT                              36  30   66      P       44   
1OX21CS142 SHREYAS BYAHATTI                      39  38   77      P       49   
1OX21CS143 SNEHA S                               33  22   55      P       45   
1OX21CS144 SRIKAR V                              47  44   91      P       50   
1OX21CS145 SRIMATHI J                            46  46   92      P       50   
1OX21CS146 SRINIVAS N                            42  23   65      P       50   
1OX21CS147 SUKSHITHA S                           46  31   77      P       50   
1OX21CS148 SUMON DAS                             32  26   58      P       50   
1OX21CS149 SUPRIYA                               42  27   69      P       50   
1OX21CS150 SUSHMA                                47  50   97      P       50   
1OX21CS152 SYED IBRAHIM                          45  46   91      P       50   
1OX21CS153 SYED MOHAMMED WAHI                    25   4   29      F       22   
1OX21CS154 TEJASWINI J                           45  32   77      P       50   
1OX21CS155 TEPPALA SNEHA                         50  42   92      P       50   

                                                          21CS42      ...  \
                                          EXT  TOT RESULT    INT EXT  ...   
USN        Name                                                       ...   
1OX21CS116 ROHAN VATS                      50  100      P     47  22  ...   
1OX21CS117 ROJA MK                         47   96      P     45  25  ...   
1OX21CS118 ROOPA R                         34   84      P     46  31  ...   
1OX21CS119 RUSHIKESH B KATTIMANI           46   96      P     37  31  ...   
1OX21CS120 S SARVESHWAR                    50  100      P     49  38  ...   
1OX21CS121 SABAH TABASUM                   50  100      P     50 

In [844]:
check.reset_index(inplace=True)
check

USN                            Name 21MATCS41                  \
                                                     INT EXT  TOT RESULT   
0   1OX21CS116                      ROHAN VATS        46  12   58      F   
1   1OX21CS117                         ROJA MK        49  43   92      P   
2   1OX21CS118                         ROOPA R        50  50  100      P   
3   1OX21CS119           RUSHIKESH B KATTIMANI        39  32   71      P   
4   1OX21CS120                    S SARVESHWAR        49  43   92      P   
5   1OX21CS121                   SABAH TABASUM        50  42   92      P   
6   1OX21CS122  SAGAR SHEKHARAGOUDA CHINTAMANI        43  32   75      P   
7   1OX21CS123                        SAHANA R        50  36   86      P   
8   1OX21CS124                    SAHEB KAMILA        42  25   67      P   
9   1OX21CS125                    SAI THILAK T        26  11   37      F   
10  1OX21CS126                  SAMIKSHA SIMMI        26   7   33      F   
11  1OX21CS127                    SAMPRIT BERA        26   0   26      F   
12  1OX21CS128                       SANDHYA S        45  39   84      P   
13  1OX21CS129                 SANIDHYA PUZARI        26   2   28      F   
14  1OX21CS130                  SANJAY SAGAR M        37  29   66      P   
15  1OX21CS131                      SANTHOSH R        47  47   94      P   
16  1OX21CS132                     SATHISH N R        27   6   33      F   
17  1OX21CS133                         SHANKAR        29   5   34      F   
18  1OX21CS134                     SHANTHINI R        50  48   98      P   
19  1OX21CS135                      SHARAN I G        37  27   64      P   
20  1OX21CS136               SHASHANK MURTHY K        31  26   57      P   
21  1OX21CS137                     SHASHANTH M        39  28   67      P   
22  1OX21CS138                     SHASHANTH R        50  42   92      P   
23  1OX21CS139                        SHEELA M        45  47   92      P   
24  1OX21CS140                          SHINDU        41  33   74      P   
25  1OX21CS141                        SHIVKANT        36  30   66      P   
26  1OX21CS142                SHREYAS BYAHATTI        39  38   77      P   
27  1OX21CS143                         SNEHA S        33  22   55      P   
28  1OX21CS144                        SRIKAR V        47  44   91      P   
29  1OX21CS145                      SRIMATHI J        46  46   92      P   
30  1OX21CS146                      SRINIVAS N        42  23   65      P   
31  1OX21CS147                     SUKSHITHA S        46  31   77      P   
32  1OX21CS148                       SUMON DAS        32  26   58      P   
33  1OX21CS149                         SUPRIYA        42  27   69      P   
34  1OX21CS150                          SUSHMA        47  50   97      P   
35  1OX21CS152                    SYED IBRAHIM        45  46   91      P   
36  1OX21CS153              SYED MOHAMMED WAHI        25   4   29      F   
37  1OX21CS154                     TEJASWINI J        45  32   77      P   
38  1OX21CS155                   TEPPALA SNEHA        50  42   92      P   

   21CSL481                  ... 21UH49            21INT49                 \
        INT EXT  TOT RESULT  ...    EXT TOT RESULT     INT EXT TOT RESULT   
0        50  50  100      P  ...     36  86      P      99   0  99      P   
1        49  47   96      P  ...     41  91      P      95   0  95      P   
2        50  34   84      P  ...     41  91      P      97   0  97      P   
3        50  46   96      P  ...     32  81      P      97   0  97      P   
4        50  50  100      P  ...     40  90      P      99   0  99      P   
5        50  50  100      P  ...     44  94      P      99   0  99      P   
6        50  49   99      P  ...     32  78      P      97   0  97      P   
7        50  50  100      P  ...     42  92      P      99   0  99      P   
8        50  50  100      P  ...     32  80      P      99   0  99      P   
9        49  45   94      P  ...     30  74    

In [845]:
data = check.values.tolist()
data

[['1OX21CS116',
  'ROHAN VATS',
  '46',
  '12',
  '58',
  'F',
  '50',
  '50',
  '100',
  'P',
  '47',
  '22',
  '69',
  'P',
  '43',
  '22',
  '65',
  'P',
  '38',
  '14',
  '52',
  'F',
  '37',
  '19',
  '56',
  'P',
  '50',
  '50',
  '100',
  'P',
  '49',
  '45',
  '94',
  'P',
  '50',
  '36',
  '86',
  'P',
  '99',
  '0',
  '99',
  'P',
  779,
  77.9,
  7.409090909090909],
 ['1OX21CS117',
  'ROJA MK',
  '49',
  '43',
  '92',
  'P',
  '49',
  '47',
  '96',
  'P',
  '45',
  '25',
  '70',
  'P',
  '49',
  '38',
  '87',
  'P',
  '48',
  '24',
  '72',
  'P',
  '46',
  '37',
  '83',
  'P',
  '50',
  '40',
  '90',
  'P',
  '48',
  '46',
  '94',
  'P',
  '50',
  '41',
  '91',
  'P',
  '95',
  '0',
  '95',
  'P',
  870,
  87.0,
  9.090909090909092],
 ['1OX21CS118',
  'ROOPA R',
  '50',
  '50',
  '100',
  'P',
  '50',
  '34',
  '84',
  'P',
  '46',
  '31',
  '77',
  'P',
  '47',
  '27',
  '74',
  'P',
  '48',
  '44',
  '92',
  'P',
  '50',
  '38',
  '88',
  'P',
  '50',
  '43',
  '93',
  'P'

In [846]:
[col[1] for col in check.columns]

['',
 '',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 'INT',
 'EXT',
 'TOT',
 'RESULT',
 '',
 '',
 '']

In [869]:
columns = check.columns.tolist()
columns

[('USN', ''),
 ('Name', ''),
 ('21MATCS41', 'INT'),
 ('21MATCS41', 'EXT'),
 ('21MATCS41', 'TOT'),
 ('21MATCS41', 'RESULT'),
 ('21CSL481', 'INT'),
 ('21CSL481', 'EXT'),
 ('21CSL481', 'TOT'),
 ('21CSL481', 'RESULT'),
 ('21CS42', 'INT'),
 ('21CS42', 'EXT'),
 ('21CS42', 'TOT'),
 ('21CS42', 'RESULT'),
 ('21CS43', 'INT'),
 ('21CS43', 'EXT'),
 ('21CS43', 'TOT'),
 ('21CS43', 'RESULT'),
 ('21CS44', 'INT'),
 ('21CS44', 'EXT'),
 ('21CS44', 'TOT'),
 ('21CS44', 'RESULT'),
 ('21BE45', 'INT'),
 ('21BE45', 'EXT'),
 ('21BE45', 'TOT'),
 ('21BE45', 'RESULT'),
 ('21CSL46', 'INT'),
 ('21CSL46', 'EXT'),
 ('21CSL46', 'TOT'),
 ('21CSL46', 'RESULT'),
 ('21KBK47', 'INT'),
 ('21KBK47', 'EXT'),
 ('21KBK47', 'TOT'),
 ('21KBK47', 'RESULT'),
 ('21UH49', 'INT'),
 ('21UH49', 'EXT'),
 ('21UH49', 'TOT'),
 ('21UH49', 'RESULT'),
 ('21INT49', 'INT'),
 ('21INT49', 'EXT'),
 ('21INT49', 'TOT'),
 ('21INT49', 'RESULT'),
 ('Total', ''),
 ('Percentage', ''),
 ('SGPA', '')]

In [853]:
cols = []
subs = []
for i in range(len(columns)):
    if i < 2 or i > (len(columns)-4):
        cols.append(columns[i][0])
    else:
        cols.append(columns[i][1])
        subs.append(columns[i][0])

In [854]:
print(cols)
print(subs)

['USN', 'Name', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'INT', 'EXT', 'TOT', 'RESULT', 'Total', 'Percentage', 'SGPA']
['21MATCS41', '21MATCS41', '21MATCS41', '21MATCS41', '21CSL481', '21CSL481', '21CSL481', '21CSL481', '21CS42', '21CS42', '21CS42', '21CS42', '21CS43', '21CS43', '21CS43', '21CS43', '21CS44', '21CS44', '21CS44', '21CS44', '21BE45', '21BE45', '21BE45', '21BE45', '21CSL46', '21CSL46', '21CSL46', '21CSL46', '21KBK47', '21KBK47', '21KBK47', '21KBK47', '21UH49', '21UH49', '21UH49', '21UH49', '21INT49', '21INT49', '21INT49', '21INT49']


In [842]:
check.columns = [col[1] for col in check.columns]
check

,,,INT,EXT,TOT,RESULT,INT,EXT,TOT,RESULT,...,EXT,TOT,RESULT,INT,EXT,TOT,RESULT,,,
0,1OX21CS116,ROHAN VATS,46,12,58,F,50,50,100,P,...,36,86,P,99,0,99,P,779,77.9,7.409091
1,1OX21CS117,ROJA MK,49,43,92,P,49,47,96,P,...,41,91,P,95,0,95,P,870,87.0,9.090909
2,1OX21CS118,ROOPA R,50,50,100,P,50,34,84,P,...,41,91,P,97,0,97,P,892,89.2,9.136364
3,1OX21CS119,RUSHIKESH B KATTIMANI,39,32,71,P,50,46,96,P,...,32,81,P,97,0,97,P,819,81.9,8.136364
4,1OX21CS120,S SARVESHWAR,49,43,92,P,50,50,100,P,...,40,90,P,99,0,99,P,897,89.7,9.363636
5,1OX21CS121,SABAH TABASUM,50,42,92,P,50,50,100,P,...,44,94,P,99,0,99,P,913,91.3,9.272727
6,1OX21CS122,SAGAR SHEKHARAGOUDA CHINTAMANI,43,32,75,P,50,49,99,P,...,32,78,P,97,0,97,P,838,83.8,8.545455
7,1OX21CS123,SAHANA R,50,36,86,P,50,50,100,P,...,42,92,P,99,0,99,P,923,92.3,9.363636
8,1OX21CS124,SAHEB KAMILA,42,25,67,P,50,50,100,P,...,32,80,P,99,0,99,P,859,85.9,8.772727
9,1OX21CS125,SAI THILAK T,26,11,37,F,49,45,94,P,...,30,74,P,95,0,95,P,744,74.4,6.681818


In [819]:
headers = [check.columns.tolist()]
headers

[['USN_', 'Name_', 'Total_', 'Percentage_', 'SGPA_']]

In [823]:
body = {'values' : headers}

In [822]:
sheets.spreadsheets().values().update(spreadsheetId='1sXIkONqU_yYWZDqsteCL_WEhrynkpTHuP6YadIvHwwY',
                                     range="A1",
                                     valueInputOption="RAW",
                                     body = body).execute()

{'spreadsheetId': '1sXIkONqU_yYWZDqsteCL_WEhrynkpTHuP6YadIvHwwY',
 'updatedRange': 'Sheet1!A1:E1',
 'updatedRows': 1,
 'updatedColumns': 5,
 'updatedCells': 5}

In [824]:
body = {'values': data}

In [825]:
sheets.spreadsheets().values().update(spreadsheetId='1sXIkONqU_yYWZDqsteCL_WEhrynkpTHuP6YadIvHwwY',
                                     range="A2",
                                     valueInputOption="RAW",
                                     body = body).execute()

{'spreadsheetId': '1sXIkONqU_yYWZDqsteCL_WEhrynkpTHuP6YadIvHwwY',
 'updatedRange': 'Sheet1!A2:E11',
 'updatedRows': 10,
 'updatedColumns': 5,
 'updatedCells': 50}

In [ ]:
check.loc[0] = 0


In [767]:
pd.concat([cols,check], ignore_index=True).reset_index(drop=True)

,0,1,2,3,4,5,6,7,8,9,...,21UH49_EXT,21UH49_TOT,21UH49_RESULT,21INT49_INT,21INT49_EXT,21INT49_TOT,21INT49_RESULT,Total_,Percentage_,SGPA_
0,USN_,Name_,21MATCS41_INT,21MATCS41_EXT,21MATCS41_TOT,21MATCS41_RESULT,21CSL481_INT,21CSL481_EXT,21CSL481_TOT,21CSL481_RESULT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36,86,P,99,0,99,P,779,77.9,7.409091
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41,91,P,95,0,95,P,870,87.0,9.090909
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41,91,P,97,0,97,P,892,89.2,9.136364
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32,81,P,97,0,97,P,819,81.9,8.136364
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40,90,P,99,0,99,P,897,89.7,9.363636
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,44,94,P,99,0,99,P,913,91.3,9.272727
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32,78,P,97,0,97,P,838,83.8,8.545455
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42,92,P,99,0,99,P,923,92.3,9.363636
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32,80,P,99,0,99,P,859,85.9,8.772727


In [755]:
check.loc[0] = check.columns

In [756]:
check

,USN_,Name_,21MATCS41_INT,21MATCS41_EXT,21MATCS41_TOT,21MATCS41_RESULT,21CSL481_INT,21CSL481_EXT,21CSL481_TOT,21CSL481_RESULT,...,21UH49_EXT,21UH49_TOT,21UH49_RESULT,21INT49_INT,21INT49_EXT,21INT49_TOT,21INT49_RESULT,Total_,Percentage_,SGPA_
0,USN_,Name_,21MATCS41_INT,21MATCS41_EXT,21MATCS41_TOT,21MATCS41_RESULT,21CSL481_INT,21CSL481_EXT,21CSL481_TOT,21CSL481_RESULT,...,21UH49_EXT,21UH49_TOT,21UH49_RESULT,21INT49_INT,21INT49_EXT,21INT49_TOT,21INT49_RESULT,Total_,Percentage_,SGPA_
1,1OX21CS117,ROJA MK,49,43,92,P,49,47,96,P,...,41,91,P,95,0,95,P,870,87.0,9.090909
2,1OX21CS118,ROOPA R,50,50,100,P,50,34,84,P,...,41,91,P,97,0,97,P,892,89.2,9.136364
3,1OX21CS119,RUSHIKESH B KATTIMANI,39,32,71,P,50,46,96,P,...,32,81,P,97,0,97,P,819,81.9,8.136364
4,1OX21CS120,S SARVESHWAR,49,43,92,P,50,50,100,P,...,40,90,P,99,0,99,P,897,89.7,9.363636
5,1OX21CS121,SABAH TABASUM,50,42,92,P,50,50,100,P,...,44,94,P,99,0,99,P,913,91.3,9.272727
6,1OX21CS122,SAGAR SHEKHARAGOUDA CHINTAMANI,43,32,75,P,50,49,99,P,...,32,78,P,97,0,97,P,838,83.8,8.545455
7,1OX21CS123,SAHANA R,50,36,86,P,50,50,100,P,...,42,92,P,99,0,99,P,923,92.3,9.363636
8,1OX21CS124,SAHEB KAMILA,42,25,67,P,50,50,100,P,...,32,80,P,99,0,99,P,859,85.9,8.772727
9,1OX21CS125,SAI THILAK T,26,11,37,F,49,45,94,P,...,30,74,P,95,0,95,P,744,74.4,6.681818


In [723]:
check.reset_index(inplace=True)

In [735]:
column_levels = [level for level in check.columns.get_level_values(0)]
column_levels

['USN',
 'Name',
 '21MATCS41',
 '21MATCS41',
 '21MATCS41',
 '21MATCS41',
 '21CSL481',
 '21CSL481',
 '21CSL481',
 '21CSL481',
 '21CS42',
 '21CS42',
 '21CS42',
 '21CS42',
 '21CS43',
 '21CS43',
 '21CS43',
 '21CS43',
 '21CS44',
 '21CS44',
 '21CS44',
 '21CS44',
 '21BE45',
 '21BE45',
 '21BE45',
 '21BE45',
 '21CSL46',
 '21CSL46',
 '21CSL46',
 '21CSL46',
 '21KBK47',
 '21KBK47',
 '21KBK47',
 '21KBK47',
 '21UH49',
 '21UH49',
 '21UH49',
 '21UH49',
 '21INT49',
 '21INT49',
 '21INT49',
 '21INT49',
 'Total',
 'Percentage',
 'SGPA']

In [741]:
headers = [f"{column_levels[i]}_{check.columns.get_level_values(1)[i]}" for i in range(len(column_levels))]
headers.insert(0, check.index.tolist())
headers

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38],
 'USN_',
 'Name_',
 '21MATCS41_INT',
 '21MATCS41_EXT',
 '21MATCS41_TOT',
 '21MATCS41_RESULT',
 '21CSL481_INT',
 '21CSL481_EXT',
 '21CSL481_TOT',
 '21CSL481_RESULT',
 '21CS42_INT',
 '21CS42_EXT',
 '21CS42_TOT',
 '21CS42_RESULT',
 '21CS43_INT',
 '21CS43_EXT',
 '21CS43_TOT',
 '21CS43_RESULT',
 '21CS44_INT',
 '21CS44_EXT',
 '21CS44_TOT',
 '21CS44_RESULT',
 '21BE45_INT',
 '21BE45_EXT',
 '21BE45_TOT',
 '21BE45_RESULT',
 '21CSL46_INT',
 '21CSL46_EXT',
 '21CSL46_TOT',
 '21CSL46_RESULT',
 '21KBK47_INT',
 '21KBK47_EXT',
 '21KBK47_TOT',
 '21KBK47_RESULT',
 '21UH49_INT',
 '21UH49_EXT',
 '21UH49_TOT',
 '21UH49_RESULT',
 '21INT49_INT',
 '21INT49_EXT',
 '21INT49_TOT',
 '21INT49_RESULT',
 'Total_',
 'Percentage_',
 'SGPA_']

In [743]:
data.insert(0, headers)
data

[[[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38],
  'USN_',
  'Name_',
  '21MATCS41_INT',
  '21MATCS41_EXT',
  '21MATCS41_TOT',
  '21MATCS41_RESULT',
  '21CSL481_INT',
  '21CSL481_EXT',
  '21CSL481_TOT',
  '21CSL481_RESULT',
  '21CS42_INT',
  '21CS42_EXT',
  '21CS42_TOT',
  '21CS42_RESULT',
  '21CS43_INT',
  '21CS43_EXT',
  '21CS43_TOT',
  '21CS43_RESULT',
  '21CS44_INT',
  '21CS44_EXT',
  '21CS44_TOT',
  '21CS44_RESULT',
  '21BE45_INT',
  '21BE45_EXT',
  '21BE45_TOT',
  '21BE45_RESULT',
  '21CSL46_INT',
  '21CSL46_EXT',
  '21CSL46_TOT',
  '21CSL46_RESULT',
  '21KBK47_INT',
  '21KBK47_EXT',
  '21KBK47_TOT',
  '21KBK47_RESULT',
  '21UH49_INT',
  '21UH49_EXT',
  '21UH49_TOT',
  '21UH49_RESULT',
  '21INT49_INT',
  '21INT49_EXT',
  '21INT49_TOT',
  '21INT49_RESULT',
  'Total_',
 

In [753]:
check.columns.to_list()

[('USN', ''),
 ('Name', ''),
 ('21MATCS41', 'INT'),
 ('21MATCS41', 'EXT'),
 ('21MATCS41', 'TOT'),
 ('21MATCS41', 'RESULT'),
 ('21CSL481', 'INT'),
 ('21CSL481', 'EXT'),
 ('21CSL481', 'TOT'),
 ('21CSL481', 'RESULT'),
 ('21CS42', 'INT'),
 ('21CS42', 'EXT'),
 ('21CS42', 'TOT'),
 ('21CS42', 'RESULT'),
 ('21CS43', 'INT'),
 ('21CS43', 'EXT'),
 ('21CS43', 'TOT'),
 ('21CS43', 'RESULT'),
 ('21CS44', 'INT'),
 ('21CS44', 'EXT'),
 ('21CS44', 'TOT'),
 ('21CS44', 'RESULT'),
 ('21BE45', 'INT'),
 ('21BE45', 'EXT'),
 ('21BE45', 'TOT'),
 ('21BE45', 'RESULT'),
 ('21CSL46', 'INT'),
 ('21CSL46', 'EXT'),
 ('21CSL46', 'TOT'),
 ('21CSL46', 'RESULT'),
 ('21KBK47', 'INT'),
 ('21KBK47', 'EXT'),
 ('21KBK47', 'TOT'),
 ('21KBK47', 'RESULT'),
 ('21UH49', 'INT'),
 ('21UH49', 'EXT'),
 ('21UH49', 'TOT'),
 ('21UH49', 'RESULT'),
 ('21INT49', 'INT'),
 ('21INT49', 'EXT'),
 ('21INT49', 'TOT'),
 ('21INT49', 'RESULT'),
 ('Total', ''),
 ('Percentage', ''),
 ('SGPA', '')]

In [729]:
headers = [f"{lvl[0]}_{lvl[1]}" for lvl in column_levels]
headers.insert(0, check.index.name)
headers

[None, '21BE45_21CS42', '_EXT']

In [730]:
data.insert(0, headers)
data

[[None, '21BE45_21CS42', '_EXT'],
 ['1OX21CS116',
  'ROHAN VATS',
  '46',
  '12',
  '58',
  'F',
  '50',
  '50',
  '100',
  'P',
  '47',
  '22',
  '69',
  'P',
  '43',
  '22',
  '65',
  'P',
  '38',
  '14',
  '52',
  'F',
  '37',
  '19',
  '56',
  'P',
  '50',
  '50',
  '100',
  'P',
  '49',
  '45',
  '94',
  'P',
  '50',
  '36',
  '86',
  'P',
  '99',
  '0',
  '99',
  'P',
  779,
  77.9,
  7.409090909090909],
 ['1OX21CS117',
  'ROJA MK',
  '49',
  '43',
  '92',
  'P',
  '49',
  '47',
  '96',
  'P',
  '45',
  '25',
  '70',
  'P',
  '49',
  '38',
  '87',
  'P',
  '48',
  '24',
  '72',
  'P',
  '46',
  '37',
  '83',
  'P',
  '50',
  '40',
  '90',
  'P',
  '48',
  '46',
  '94',
  'P',
  '50',
  '41',
  '91',
  'P',
  '95',
  '0',
  '95',
  'P',
  870,
  87.0,
  9.090909090909092],
 ['1OX21CS118',
  'ROOPA R',
  '50',
  '50',
  '100',
  'P',
  '50',
  '34',
  '84',
  'P',
  '46',
  '31',
  '77',
  'P',
  '47',
  '27',
  '74',
  'P',
  '48',
  '44',
  '92',
  'P',
  '50',
  '38',
  '88',
  

In [725]:
headers = [check.index.name] + list(check.columns)
headers

[None,
 ('USN', ''),
 ('Name', ''),
 ('21MATCS41', 'INT'),
 ('21MATCS41', 'EXT'),
 ('21MATCS41', 'TOT'),
 ('21MATCS41', 'RESULT'),
 ('21CSL481', 'INT'),
 ('21CSL481', 'EXT'),
 ('21CSL481', 'TOT'),
 ('21CSL481', 'RESULT'),
 ('21CS42', 'INT'),
 ('21CS42', 'EXT'),
 ('21CS42', 'TOT'),
 ('21CS42', 'RESULT'),
 ('21CS43', 'INT'),
 ('21CS43', 'EXT'),
 ('21CS43', 'TOT'),
 ('21CS43', 'RESULT'),
 ('21CS44', 'INT'),
 ('21CS44', 'EXT'),
 ('21CS44', 'TOT'),
 ('21CS44', 'RESULT'),
 ('21BE45', 'INT'),
 ('21BE45', 'EXT'),
 ('21BE45', 'TOT'),
 ('21BE45', 'RESULT'),
 ('21CSL46', 'INT'),
 ('21CSL46', 'EXT'),
 ('21CSL46', 'TOT'),
 ('21CSL46', 'RESULT'),
 ('21KBK47', 'INT'),
 ('21KBK47', 'EXT'),
 ('21KBK47', 'TOT'),
 ('21KBK47', 'RESULT'),
 ('21UH49', 'INT'),
 ('21UH49', 'EXT'),
 ('21UH49', 'TOT'),
 ('21UH49', 'RESULT'),
 ('21INT49', 'INT'),
 ('21INT49', 'EXT'),
 ('21INT49', 'TOT'),
 ('21INT49', 'RESULT'),
 ('Total', ''),
 ('Percentage', ''),
 ('SGPA', '')]

In [709]:
data.insert(0,headers)
data

[[MultiIndex([('1OX21CS116',                     'ROHAN VATS'),
              ('1OX21CS117',                        'ROJA MK'),
              ('1OX21CS118',                        'ROOPA R'),
              ('1OX21CS119',          'RUSHIKESH B KATTIMANI'),
              ('1OX21CS120',                   'S SARVESHWAR'),
              ('1OX21CS121',                  'SABAH TABASUM'),
              ('1OX21CS122', 'SAGAR SHEKHARAGOUDA CHINTAMANI'),
              ('1OX21CS123',                       'SAHANA R'),
              ('1OX21CS124',                   'SAHEB KAMILA'),
              ('1OX21CS125',                   'SAI THILAK T'),
              ('1OX21CS126',                 'SAMIKSHA SIMMI'),
              ('1OX21CS127',                   'SAMPRIT BERA'),
              ('1OX21CS128',                      'SANDHYA S'),
              ('1OX21CS129',                'SANIDHYA PUZARI'),
              ('1OX21CS130',                 'SANJAY SAGAR M'),
              ('1OX21CS131',            

In [887]:
import smtplib
from email.message import EmailMessage

In [894]:
sender_email = "srikarvuchiha@gmail.com"
receiver_email = "vsrikar44@gmail.com"
password = "glhy xvwc uaxs rstn"
sheetLink = "https://docs.google.com/spreadsheets/d/1vlMC3HTQHjOz7koN86cHFrv4_um63-nfhOOd9jy5FXk"
message = EmailMessage()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = "Checking if mail sending programmatically works"
message.set_content(f"Here is the spreadsheets with the result extracted Access it here: {sheetLink}")

In [895]:
# Connect to the SMTP server
with smtplib.SMTP("smtp.gmail.com", 587) as server:
    server.starttls()
    server.login(sender_email, password)
    # Send email
    server.send_message(message)

print("Email sent successfully")

Email sent successfully


In [907]:
students_marks.sort_values(by=['Percentage', 'SGPA'], ascending=False)[:10][['Total', 'Percentage', 'SGPA']]

,,Total,Percentage,SGPA
,,,,
USN,Name,,,
1OX21CS123,SAHANA R,923,92.3,9.363636
1OX21CS145,SRIMATHI J,915,91.5,9.545455
1OX21CS152,SYED IBRAHIM,913,91.3,9.5
1OX21CS121,SABAH TABASUM,913,91.3,9.272727
1OX21CS155,TEPPALA SNEHA,911,91.1,9.545455
1OX21CS120,S SARVESHWAR,897,89.7,9.363636
1OX21CS138,SHASHANTH R,896,89.6,9.227273
1OX21CS128,SANDHYA S,894,89.4,9.227273
